In [7]:
#from the site : https://docs.python-guide.org/scenarios/scrape/
import urllib
#import requests
import bs4
import re
import cobra

In [2]:
#load in the most recent version of the metabolic model
model = cobra.io.read_sbml_model("../results/reconstructions/psy_cor_6.xml")
pa14 = cobra.io.read_sbml_model("../results/figures/figure1/comp_other_species/iPAU1129_pa14.xml")
pao1 = cobra.io.read_sbml_model("../results/figures/figure1/comp_other_species/iPAE1146_pa01.xml")
k12 = cobra.io.read_sbml_model("../results/figures/figure1/comp_other_species/iML1515_ecoli.xml")

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy,information is not parsed
SBML package 'render' not supported by cobrapy,information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_cpd00015_c "FAD">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_cpd03128_c "Decanoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_cpd00982_c "FADH2">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_cpd03129_c "(2E)-Decenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_cpd01260_c "Lauroyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_cpd02060_c "(2E)-Dodecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemical

In [3]:
#generate list of genes in the model
gene_list = []
for x in model.genes:
    gene_list.append(x.id)

In [3]:
#generate list of PSPTO numbers, all four digits, starting at 0001
v = [str(i) for i in range (1,10)]
c = [str(i) for i in range (10,100)]
m = [str(i) for i in range(100,999)]
l = [str(i) for i in range(1000,9999)]

#generate a sequential numbering with the proper four digit numbering code, used because PSPTO_0001 not PSPTO_1
num_list = []
for x in v:
    x = "000" + x
    num_list.append(x) 
for x in c:
    x = "00" + x
    num_list.append(x)
for x in m:
    x = "0" + x
    num_list.append(x)
for x in l:
    num_list.append(x)
    


In [5]:
#use this script to scrape the initial results from kegg, this is written for specifically the genes in the iPTO19 model
GO_dict = {}
url = "https://www.genome.jp/dbget-bin/www_bget?pst:"
for x in gene_list:
    url_full = url + str(x)
    opened_url = urllib.request.urlopen(url_full)
    soup = bs4.BeautifulSoup(opened_url)
#print (soup)
    table = soup.find("table")
    if table != None:
        headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  # print (headings)
        for row in table.find_all("tr")[1:]:
            dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
   #print (dataset)
            if "Pathway" in dataset:
       #datasets.append(dataset)
                for key, value in dataset.items():
                    if "Pathway" in value:
           #print (value)
                        result = re.search('(?s)Pathway(.*)Module', value)
                        if result:
                            GO_dict[x] = result.groups()
                           #print(result.groups())
                        result1 = re.search('(?s)Pathway(.*)Brite', value)
                        if result1:
                            GO_dict[x] = result1.groups()
                            #print(result1.groups())
                    else: pass
    else: print ("========I DID NOT FIND A TABLE FOR========== :: " + x)

========I DID NOT FIND A TABLE FOR========== :: PSTPO_0757
========I DID NOT FIND A TABLE FOR========== :: PSTPO_0324
========I DID NOT FIND A TABLE FOR========== :: Unknown
========I DID NOT FIND A TABLE FOR========== :: PSPTO_5728
========I DID NOT FIND A TABLE FOR========== :: SPONTANEOUS
========I DID NOT FIND A TABLE FOR========== :: PSTPO_3833
========I DID NOT FIND A TABLE FOR========== :: PSTPO_5279
========I DID NOT FIND A TABLE FOR========== :: PAPTO_2706


In [6]:
#extract all the proper headings from the dictionary using re.split, which deletes the re object and splits at each instance of it
#this should give me only the pathway titles
a = {}

for x, y in GO_dict.items():
    for m in y:
        result = re.split('\\xa0\\xa0', m)
        b = []        
        for go in result:
            result1 = re.split('pst', go)
            c = []
            for term in result1:
                result2 = re.split('\\n' , term)
                #print (result2 , " is to " , x)
                c.append(result2[:])
            b.append(c)
            #print ("The list b is :",  b)
            #print (b , x)
        a[x] = b
    print ("I have added",x , "to the dictionary")

I have added PSPTO_0722 to the dictionary
I have added PSPTO_1361 to the dictionary
I have added PSPTO_0982 to the dictionary
I have added PSPTO_3860 to the dictionary
I have added PSPTO_0981 to the dictionary
I have added PSPTO_5005 to the dictionary
I have added PSPTO_3862 to the dictionary
I have added PSPTO_4921 to the dictionary
I have added PSPTO_4290 to the dictionary
I have added PSPTO_5251 to the dictionary
I have added PSPTO_0691 to the dictionary
I have added PSPTO_3902 to the dictionary
I have added PSPTO_0128 to the dictionary
I have added PSPTO_0230 to the dictionary
I have added PSPTO_3793 to the dictionary
I have added PSPTO_2106 to the dictionary
I have added PSPTO_5121 to the dictionary
I have added PSPTO_4254 to the dictionary
I have added PSPTO_0549 to the dictionary
I have added PSPTO_1509 to the dictionary
I have added PSPTO_0923 to the dictionary
I have added PSPTO_4895 to the dictionary
I have added PSPTO_4891 to the dictionary
I have added PSPTO_4894 to the dic

In [7]:
#I want to join all the lists that appear within the dictionary value for a 
#found this on stackoverflow : https://stackoverflow.com/questions/716477/join-list-of-lists-in-python
#recreated a new dictionary with the joined lists and the proper keys
#this needs to be done with the kegg ontology of the whole genome as well

joined_model_ont = {}
for key, value in a.items():
    value1 = [j for i in value for j in i]
    value2 = [m for n in value1 for m in n]
    #print (value2)
    joined_model_ont[key] = value2
print (joined_model_ont)

{'PSPTO_0722': ['', '', '00190', 'Oxidative phosphorylation', ''], 'PSPTO_1361': ['', '', '00220', 'Arginine biosynthesis', '00791', 'Atrazine degradation', '01100', 'Metabolic pathways', '01120', 'Microbial metabolism in diverse environments', ''], 'PSPTO_0982': ['', '', '00290', 'Valine, leucine and isoleucine biosynthesis', '00650', 'Butanoate metabolism', '00660', 'C5-Branched dibasic acid metabolism', '00770', 'Pantothenate and CoA biosynthesis', '01100', 'Metabolic pathways', '01110', 'Biosynthesis of secondary metabolites', '01130', 'Biosynthesis of antibiotics', '01210', '2-Oxocarboxylic acid metabolism', '01230', 'Biosynthesis of amino acids', 'Module', '', '_M00019', 'Valine/isoleucine biosynthesis, pyruvate => valine / 2-oxobutanoate => isoleucine', '_M00570', 'Isoleucine biosynthesis, threonine => 2-oxobutanoate => isoleucine', ''], 'PSPTO_3860': ['', '', '00010', 'Glycolysis / Gluconeogenesis', '00020', 'Citrate cycle (TCA cycle)', '00620', 'Pyruvate metabolism', '01100', 

In [8]:
#generate the numbers of genes that fall into the different ontologies
#first within the genes of the model
#a is the dictionary that contains the pertinant genes for the model

#initiate a list of terms to be extracted from the dictionary, this will ensure that only terms present in the list will be used as categories for the genes to actually fall into
go_terms = []
for key,value in joined_model_ont.items():
    for term in value:
        if term in go_terms:
            pass
        else:
            go_terms.append(term)
#print (go_terms)

#now genreate the list of genes that correspond to each go term
#finally, count the number of members in each list, this is the number of genes that has a distinct ontology.
go_term_numbers= {}
for term in go_terms:
    #print (term , "=========================")
    go_list = []
    for key, value in joined_model_ont.items():
        #print(value)

        for terms in value:
            #print (terms)

            if term == terms:
                go_list.append(key)
            else:
                pass
    go_term_numbers[term] = len(go_list)
    #print (len(go_list), term)

In [9]:
#generate a dataframe with the ontologies and the number of genes that fit in each, tehn write out to CSV
#print (go_term_numbers)
import pandas as pd
ipto_df = pd.DataFrame.from_dict(go_term_numbers, orient = 'index')
ipto_df.to_csv("../data/ipto_gene_ontologies_figure_1.csv")

url = "https://www.genome.jp/dbget-bin/www_bget?pst:PSPTO_0004"

opened_url = urllib.request.urlopen(url)
soup = bs4.BeautifulSoup(opened_url)
#print (soup)
table = soup.find("table")
headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
print (headings)
for row in table.find_all("tr")[1:]:
    dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
    if "Pathway" in dataset:
        #datasets.append(dataset)
        for key, value in dataset.items():
            if "Pathway" in value:
            #print (value)
                result = re.search('(?s)Pathway(.*)Module', value)
                if result:
                    print(result.groups())
                result1 = re.search('(?s)Pathway(.*)Brite', value)
                if result1:
                    print(result1.groups())
            else: pass


In [4]:
#this is to gather information from the kegg website, using the number list generated in cell 4. 
GO_dict_kegg = {}
url = "https://www.genome.jp/dbget-bin/www_bget?pst:PSPTO_"
for x in num_list:
    url_full = url + str(x)
    opened_url = urllib.request.urlopen(url_full)
    soup = bs4.BeautifulSoup(opened_url)
#print (soup)
    table = soup.find("table")
    if table != None:
        headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  # print (headings)
        for row in table.find_all("tr")[1:]:
            dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
   #print (dataset)
            if "Pathway" in dataset:
       #datasets.append(dataset)
                for key, value in dataset.items():
                    if "Pathway" in value:
           #print (value)
                        result = re.search('(?s)Pathway(.*)Module', value)
                        if result:
                            GO_dict_kegg[x] = result.groups()
                           #print(result.groups())
                        result1 = re.search('(?s)Pathway(.*)Brite', value)
                        if result1:
                            GO_dict_kegg[x] = result1.groups()
                            #print(result1.groups())
                    else: pass
    else: print ("========I DID NOT FIND A TABLE FOR========== :: " + x)

========I DID NOT FIND A TABLE FOR========== :: 0010
========I DID NOT FIND A TABLE FOR========== :: 0122
========I DID NOT FIND A TABLE FOR========== :: 0160
========I DID NOT FIND A TABLE FOR========== :: 0195
========I DID NOT FIND A TABLE FOR========== :: 0198
========I DID NOT FIND A TABLE FOR========== :: 0199
========I DID NOT FIND A TABLE FOR========== :: 0204
========I DID NOT FIND A TABLE FOR========== :: 0392
========I DID NOT FIND A TABLE FOR========== :: 0504
========I DID NOT FIND A TABLE FOR========== :: 0590
========I DID NOT FIND A TABLE FOR========== :: 0839
========I DID NOT FIND A TABLE FOR========== :: 0843
========I DID NOT FIND A TABLE FOR========== :: 0845
========I DID NOT FIND A TABLE FOR========== :: 0870
========I DID NOT FIND A TABLE FOR========== :: 0872
========I DID NOT FIND A TABLE FOR========== :: 1063
========I DID NOT FIND A TABLE FOR========== :: 1113
========I DID NOT FIND A TABLE FOR========== :: 1442
========I DID NOT FIND A TABLE FOR========== :

URLError: <urlopen error [Errno 60] Operation timed out>

In [5]:
#generate a dictionary for teh kegg terms

kegg_ont = {}

for x, y in GO_dict_kegg.items():
    for m in y:
        result = re.split('\\xa0\\xa0', m)
        kegg_b = []        
        for go in result:
            result1 = re.split('pst', go)
            kegg_c = []
            for term in result1:
                result2 = re.split('\\n' , term)
                #print (result2 , " is to " , x)
                kegg_c.append(result2[:])
            kegg_b.append(kegg_c)
            #print ("The list b is :",  b)
            #print (b , x)
        kegg_ont[x] = kegg_b
    print ("I have added", x , "to the dictionary")

I have added 0001 to the dictionary
I have added 0002 to the dictionary
I have added 0003 to the dictionary
I have added 0065 to the dictionary
I have added 0073 to the dictionary
I have added 0074 to the dictionary
I have added 0075 to the dictionary
I have added 0079 to the dictionary
I have added 0080 to the dictionary
I have added 0082 to the dictionary
I have added 0083 to the dictionary
I have added 0084 to the dictionary
I have added 0085 to the dictionary
I have added 0087 to the dictionary
I have added 0089 to the dictionary
I have added 0090 to the dictionary
I have added 0092 to the dictionary
I have added 0095 to the dictionary
I have added 0101 to the dictionary
I have added 0103 to the dictionary
I have added 0109 to the dictionary
I have added 0112 to the dictionary
I have added 0117 to the dictionary
I have added 0125 to the dictionary
I have added 0126 to the dictionary
I have added 0127 to the dictionary
I have added 0128 to the dictionary
I have added 0129 to the dic

In [6]:
#I want to join all the lists that appear within the dictionary value for a 
#found this on stackoverflow : https://stackoverflow.com/questions/716477/join-list-of-lists-in-python
#recreated a new dictionary with the joined lists and the proper keys
#this needs to be done with the kegg ontology of the whole genome as well

joined_kegg_ont = {}
for key, value in kegg_ont.items():
    value1 = [j for i in value for j in i]
    value2 = [m for n in value1 for m in n]
    #print (value2)
    joined_kegg_ont["PSPTO_" + str(key)] = value2
print (joined_kegg_ont)

{'PSPTO_0001': ['', '', '02020', 'Two-component system', 'Brite', 'KEGG Orthology (KO) [BR:', '00001]', '\xa009130 Environmental Information Processing', '', '09132 Signal transduction', '', '\xa002020 Two-component system', '', '', 'PSPTO_0001 (dnaA)', '\xa009180 '], 'PSPTO_0002': ['', '', '03030', 'DNA replication', '03430', 'Mismatch repair', '03440', 'Homologous recombination', 'Module', '', '_M00260', 'DNA polymerase III complex, bacteria', 'Brite', 'KEGG Orthology (KO) [BR:', '00001]', '\xa009120 Genetic Information Processing', '', '09124 Replication and repair', '', '\xa003030 DNA replication', '', '', 'PSPTO_0002 (dnaN)', '', '\xa003430 Mismatch repair', '', '', 'PSPTO_0002 (dnaN)', '', '\xa003440 Homologous recombination', '', '', 'PSPTO_0002 (dnaN)', '\xa009180 '], 'PSPTO_0003': ['', '', '03440', 'Homologous recombination', 'Brite', 'KEGG Orthology (KO) [BR:', '00001]', '\xa009120 Genetic Information Processing', '', '09124 Replication and repair', '', '\xa003440 Homologous 

In [32]:
#generate the numbers of genes that fall into the different ontologies
#now within the kegg ontologies for the whole genome
#a is the dictionary that contains the pertinant genes for the model

#initiate a list of terms to be extracted from the dictionary, this will ensure that only terms present in the list will be used as categories for the genes to actually fall into
go_terms_kegg = []
for key,value in joined_kegg_ont.items():
    #print (value)
    for term in value:
        #print (term)
        if term in go_terms_kegg:
            pass
        else:
            go_terms_kegg.append(term)
            
#print (go_terms_kegg)

#now genreate the list of genes that correspond to each go term
#finally, count the number of members in each list, this is the number of genes that has a distinct ontology.
go_term_numbers_kegg= {}
for term in go_terms_kegg:
    #print (term , "=========================")
    go_list_each_ont = []
    for key, value in joined_kegg_ont.items():
        #print(value)

        if term in value:
            #print (term, key)
            go_list_each_ont.append(key)
                #print('yes daddy')
        else:
            pass
    #print (go_list_each_ont)
    go_term_numbers_kegg[term] = len(go_list_each_ont)
print (go_term_numbers_kegg)

{'': 1307, '02020': 158, 'Two-component system': 158, 'Brite': 716, 'KEGG Orthology (KO) [BR:': 716, '00001]': 716, '\xa009130 Environmental Information Processing': 350, '09132 Signal transduction': 134, '\xa002020 Two-component system': 134, 'PSPTO_0001 (dnaA)': 1, '\xa009180 ': 715, '03030': 15, 'DNA replication': 15, '03430': 17, 'Mismatch repair': 17, '03440': 21, 'Homologous recombination': 21, 'Module': 302, '_M00260': 7, 'DNA polymerase III complex, bacteria': 7, '\xa009120 Genetic Information Processing': 146, '09124 Replication and repair': 44, '\xa003030 DNA replication': 15, 'PSPTO_0002 (dnaN)': 1, '\xa003430 Mismatch repair': 17, '\xa003440 Homologous recombination': 21, 'PSPTO_0003 (recF)': 1, 'PSPTO_0065 (recG)': 1, '00230': 51, 'Purine metabolism': 51, '01100': 657, 'Metabolic pathways': 657, '\xa009100 Metabolism': 161, '09104 Nucleotide metabolism': 12, '\xa000230 Purine metabolism': 11, 'PSPTO_0073 (spoT)': 1, '03020': 4, 'RNA polymerase': 4, '_M00183': 4, 'RNA polym

In [35]:
#generate a dataframe with the ontologies and the number of genes that fit in each, tehn write out to CSV
#print (go_term_numbers)
import pandas as pd
kegg_pto_df = pd.DataFrame.from_dict(go_term_numbers_kegg, orient = 'index')
print (kegg_pto_df)
kegg_pto_df.to_csv("../data/kegg_pto_gene_ontologies_figure_1.csv")

                                                0
                                             1307
02020                                         158
Two-component system                          158
Brite                                         716
KEGG Orthology (KO) [BR:                      716
00001]                                        716
 09130 Environmental Information Processing   350
09132 Signal transduction                     134
 02020 Two-component system                   134
PSPTO_0001 (dnaA)                               1
 09180                                        715
03030                                          15
DNA replication                                15
03430                                          17
Mismatch repair                                17
03440                                          21
Homologous recombination                       21
Module                                        302
_M00260                                         7


In [14]:
#pa14 uses kegg ids, https://www.genome.jp/dbget-bin/www_bget?pau:PA14_00290
#pao1 uses kegg ids, https://www.genome.jp/dbget-bin/www_bget?pae:PA0027
#k12 uses bigg ids, but they correlate with kegg, https://www.genome.jp/dbget-bin/www_bget?eco:b0002

#similar to cell below, where gene lists are generated for all of the genes within ipto, will do same thing for three above
pa14_genes = []
for f in pa14.genes:
    pa14_genes.append(f.id)
    
pao1_genes = []
for f in pao1.genes:
    pao1_genes.append(f.id)
    
k12_genes = []
for f in k12.genes:
    k12_genes.append(f.id)
    


In [16]:
print (pa14_genes)

['PA14_54670', 'PA14_54450', 'PA14_48020', 'PA14_45430', 'PA14_43680', 'PA14_43640', 'PA14_41170', 'PA14_38440', 'PA14_25660', 'PA14_25390', 'PA14_23790', 'PA14_22890', 'PA14_23450', 'PA14_00250', 'PA14_00290', 'PA14_03050', 'PA14_03430', 'PA14_04110', 'PA14_04580', 'PA14_05150', 'PA14_05590', 'PA14_05840', 'PA14_07170', 'PA14_07740', 'PA14_08480', 'PA14_54170', 'PA14_53950', 'PA14_52900', 'PA14_52670', 'PA14_52270', 'PA14_46470', 'PA14_44470', 'PA14_43970', 'PA14_43690', 'PA14_41350', 'PA14_38590', 'PA14_38330', 'PA14_35320', 'PA14_34970', 'PA14_34360', 'PA14_33810', 'PA14_30630', 'PA14_30340', 'PA14_30190', 'PA14_26460', 'PA14_25550', 'PA14_24220', 'PA14_23310', 'PA14_20140', 'PA14_19870', 'PA14_19470', 'PA14_61400', 'PA14_18580', 'PA14_17400', 'PA14_17130', 'PA14_16360', 'PA14_16070', 'PA14_15310', 'PA14_14880', 'PA14_13500', 'PA14_12020', 'PA14_12010', 'PA14_11400', 'PA14_10630', 'PA14_10260', 'PA14_10250', 'PA14_10240', 'PA14_10230', 'PA14_09400', 'PA14_57160', 'PA14_57780', 'PA14

In [17]:
#for pa14

pa14GO_dict = {}
url = "https://www.genome.jp/dbget-bin/www_bget?pau:"
for x in pa14_genes:
    url_full = url + str(x)
    opened_url = urllib.request.urlopen(url_full)
    soup = bs4.BeautifulSoup(opened_url)
#print (soup)
    table = soup.find("table")
    if table != None:
        headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  # print (headings)
        for row in table.find_all("tr")[1:]:
            dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
   #print (dataset)
            if "Pathway" in dataset:
       #datasets.append(dataset)
                for key, value in dataset.items():
                    if "Pathway" in value:
           #print (value)
                        result = re.search('(?s)Pathway(.*)Module', value)
                        if result:
                            pa14GO_dict[x] = result.groups()
                           #print(result.groups())
                        result1 = re.search('(?s)Pathway(.*)Brite', value)
                        if result1:
                            pa14GO_dict[x] = result1.groups()
                            #print(result1.groups())
                    else: pass
    else: print ("========I DID NOT FIND A TABLE FOR========== :: " + x)

========I DID NOT FIND A TABLE FOR========== :: unassigned
========I DID NOT FIND A TABLE FOR========== :: SPONTANEOUS
========I DID NOT FIND A TABLE FOR========== :: Unassigned


In [26]:
#extract all the proper headings from the dictionary using re.split, which deletes the re object and splits at each instance of it
#this should give me only the pathway titles
pa14splits = {}

for x, y in pa14GO_dict.items():
    for m in y:
        result = re.split('\\xa0\\xa0', m)
        b = []        
        for go in result:
            #change the split requirements based on the organism in use, should be the same as the three letter code in the url
            result1 = re.split('pau', go)
            c = []
            for term in result1:
                result2 = re.split('\\n' , term)
                #print (result2 , " is to " , x)
                c.append(result2[:])
            b.append(c)
            #print ("The list b is :",  b)
            #print (b , x)
        pa14splits[x] = b
    print ("I have added",x , "to the dictionary")

I have added PA14_54450 to the dictionary
I have added PA14_48020 to the dictionary
I have added PA14_45430 to the dictionary
I have added PA14_43680 to the dictionary
I have added PA14_43640 to the dictionary
I have added PA14_41170 to the dictionary
I have added PA14_38440 to the dictionary
I have added PA14_25660 to the dictionary
I have added PA14_25390 to the dictionary
I have added PA14_23790 to the dictionary
I have added PA14_22890 to the dictionary
I have added PA14_00290 to the dictionary
I have added PA14_03050 to the dictionary
I have added PA14_03430 to the dictionary
I have added PA14_04110 to the dictionary
I have added PA14_04580 to the dictionary
I have added PA14_05150 to the dictionary
I have added PA14_05590 to the dictionary
I have added PA14_05840 to the dictionary
I have added PA14_07170 to the dictionary
I have added PA14_07740 to the dictionary
I have added PA14_08480 to the dictionary
I have added PA14_54170 to the dictionary
I have added PA14_53950 to the dic

In [27]:
#first join all the lists together if there is many in the values of pa14splits
pa14splits_joined = {}
for key, value in pa14splits.items():
    value1 = [j for i in value for j in i]
    value2 = [m for n in value1 for m in n]
    #print (value2)
    pa14splits_joined[key] = value2
#generate the numbers of genes that fall into the different ontologies
#now within the kegg ontologies for the pa14 model

#initiate a list of terms to be extracted from the dictionary, this will ensure that only terms present in the list will be used as categories for the genes to actually fall into
pa14go_terms_kegg = []
for key,value in pa14splits_joined.items():
    #print (value)
    for term in value:
        #print (term)
        if term in pa14go_terms_kegg:
            pass
        else:
            pa14go_terms_kegg.append(term)
            
#print (go_terms_kegg)

#now genreate the list of genes that correspond to each go term
#finally, count the number of members in each list, this is the number of genes that has a distinct ontology.
pa14go_term_numbers_kegg= {}
for term in pa14go_terms_kegg:
    #print (term , "=========================")
    pa14go_list_each_ont = []
    for key, value in pa14splits_joined.items():
        #print(value)

        if term in value:
            #print (term, key)
            pa14go_list_each_ont.append(key)
                #print('yes daddy')
        else:
            pass
    #print (go_list_each_ont)
    pa14go_term_numbers_kegg[term] = len(pa14go_list_each_ont)
print (pa14go_term_numbers_kegg)

{'': 427, '00250': 16, 'Alanine, aspartate and glutamate metabolism': 16, '00760': 10, 'Nicotinate and nicotinamide metabolism': 10, '01100': 391, 'Metabolic pathways': 391, 'Module': 170, '_M00115': 5, 'NAD biosynthesis, aspartate => NAD': 5, '00310': 8, 'Lysine degradation': 8, '00061': 11, 'Fatty acid biosynthesis': 11, '00780': 9, 'Biotin metabolism': 9, '01130': 117, 'Biosynthesis of antibiotics': 117, '01212': 15, 'Fatty acid metabolism': 15, '_M00083': 5, 'Fatty acid biosynthesis, elongation': 5, '_M00572': 4, 'Pimeloyl-ACP biosynthesis, BioC-BioH pathway, malonyl-ACP => pimeloyl-ACP': 4, 'Brite': 120, 'KEGG Orthology (KO) [BR:': 120, '00001]': 120, '\xa009100 Metabolism': 102, '09103 Lipid metabolism': 10, '\xa000061 Fatty acid biosynthesis': 7, 'PA14_45430': 1, '09108 Metabolism of cofactors and vitamins': 19, '\xa000780 Biotin metabolism': 7, '09110 Biosynthesis of other secondary metabolites': 5, '\xa000333 Prodigiosin biosynthesis': 4, '\xa009180 ': 120, 'PA14_43680 (fabA)'

In [18]:
pao1GO_dict = {}
url = "https://www.genome.jp/dbget-bin/www_bget?pae:"
for x in pao1_genes:
    url_full = url + str(x)
    opened_url = urllib.request.urlopen(url_full)
    soup = bs4.BeautifulSoup(opened_url)
#print (soup)
    table = soup.find("table")
    if table != None:
        headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  # print (headings)
        for row in table.find_all("tr")[1:]:
            dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
   #print (dataset)
            if "Pathway" in dataset:
       #datasets.append(dataset)
                for key, value in dataset.items():
                    if "Pathway" in value:
           #print (value)
                        result = re.search('(?s)Pathway(.*)Module', value)
                        if result:
                            pao1GO_dict[x] = result.groups()
                           #print(result.groups())
                        result1 = re.search('(?s)Pathway(.*)Brite', value)
                        if result1:
                            pao1GO_dict[x] = result1.groups()
                            #print(result1.groups())
                    else: pass
    else: print ("========I DID NOT FIND A TABLE FOR========== :: " + x)

========I DID NOT FIND A TABLE FOR========== :: SPONTANEOUS
========I DID NOT FIND A TABLE FOR========== :: Unassigned


In [28]:
pao1splits = {}

for x, y in pao1GO_dict.items():
    for m in y:
        result = re.split('\\xa0\\xa0', m)
        b = []        
        for go in result:
            #change the split requirements based on the organism in use, should be the same as the three letter code in the ur
            result1 = re.split('pae', go)
            c = []
            for term in result1:
                result2 = re.split('\\n' , term)
                #print (result2 , " is to " , x)
                c.append(result2[:])
            b.append(c)
            #print ("The list b is :",  b)
            #print (b , x)
        pao1splits[x] = b
    print ("I have added",x , "to the dictionary")

I have added PA0761 to the dictionary
I have added PA1252 to the dictionary
I have added PA1470 to the dictionary
I have added PA1610 to the dictionary
I have added PA1614 to the dictionary
I have added PA1806 to the dictionary
I have added PA2015 to the dictionary
I have added PA2967 to the dictionary
I have added PA2991 to the dictionary
I have added PA3118 to the dictionary
I have added PA3195 to the dictionary
I have added PA0025 to the dictionary
I have added PA0247 to the dictionary
I have added PA0265 to the dictionary
I have added PA0316 to the dictionary
I have added PA0350 to the dictionary
I have added PA0393 to the dictionary
I have added PA0430 to the dictionary
I have added PA0447 to the dictionary
I have added PA0551 to the dictionary
I have added PA0593 to the dictionary
I have added PA0662 to the dictionary
I have added PA0782 to the dictionary
I have added PA0795 to the dictionary
I have added PA0879 to the dictionary
I have added PA0898 to the dictionary
I have added

In [29]:
#first join all the lists together if there is many in the values of pa14splits
pao1splits_joined = {}
for key, value in pao1splits.items():
    value1 = [j for i in value for j in i]
    value2 = [m for n in value1 for m in n]
    #print (value2)
    pao1splits_joined[key] = value2
#generate the numbers of genes that fall into the different ontologies
#now within the kegg ontologies for the pa14 model

#initiate a list of terms to be extracted from the dictionary, this will ensure that only terms present in the list will be used as categories for the genes to actually fall into
pao1go_terms_kegg = []
for key,value in pao1splits_joined.items():
    #print (value)
    for term in value:
        #print (term)
        if term in pao1go_terms_kegg:
            pass
        else:
            pao1go_terms_kegg.append(term)
            
#print (go_terms_kegg)

#now genreate the list of genes that correspond to each go term
#finally, count the number of members in each list, this is the number of genes that has a distinct ontology.
pao1go_term_numbers_kegg= {}
for term in pao1go_terms_kegg:
    #print (term , "=========================")
    pao1go_list_each_ont = []
    for key, value in pao1splits_joined.items():
        #print(value)

        if term in value:
            #print (term, key)
            pao1go_list_each_ont.append(key)
                #print('yes daddy')
        else:
            pass
    #print (go_list_each_ont)
    pao1go_term_numbers_kegg[term] = len(pao1go_list_each_ont)
print (pao1go_term_numbers_kegg)

{'': 424, '00250': 16, 'Alanine, aspartate and glutamate metabolism': 16, '00760': 10, 'Nicotinate and nicotinamide metabolism': 10, '01100': 388, 'Metabolic pathways': 388, 'Module': 161, '_M00115': 5, 'NAD biosynthesis, aspartate => NAD': 5, '00310': 8, 'Lysine degradation': 8, '00061': 11, 'Fatty acid biosynthesis': 11, '00780': 9, 'Biotin metabolism': 9, '01130': 116, 'Biosynthesis of antibiotics': 116, '01212': 15, 'Fatty acid metabolism': 15, '_M00083': 5, 'Fatty acid biosynthesis, elongation': 5, '_M00572': 4, 'Pimeloyl-ACP biosynthesis, BioC-BioH pathway, malonyl-ACP => pimeloyl-ACP': 4, 'Brite': 123, 'KEGG Orthology (KO) [BR:': 123, '00001]': 123, '\xa009100 Metabolism': 105, '09103 Lipid metabolism': 10, '\xa000061 Fatty acid biosynthesis': 7, 'PA1470': 1, '09108 Metabolism of cofactors and vitamins': 19, '\xa000780 Biotin metabolism': 7, '09110 Biosynthesis of other secondary metabolites': 5, '\xa000333 Prodigiosin biosynthesis': 4, '\xa009180 ': 123, 'PA1610 (fabA)': 1, '00

In [19]:
k12GO_dict = {}
url = "https://www.genome.jp/dbget-bin/www_bget?eco:"
for x in k12_genes:
    url_full = url + str(x)
    opened_url = urllib.request.urlopen(url_full)
    soup = bs4.BeautifulSoup(opened_url)
#print (soup)
    table = soup.find("table")
    if table != None:
        headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  # print (headings)
        for row in table.find_all("tr")[1:]:
            dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
   #print (dataset)
            if "Pathway" in dataset:
       #datasets.append(dataset)
                for key, value in dataset.items():
                    if "Pathway" in value:
           #print (value)
                        result = re.search('(?s)Pathway(.*)Module', value)
                        if result:
                            k12GO_dict[x] = result.groups()
                           #print(result.groups())
                        result1 = re.search('(?s)Pathway(.*)Brite', value)
                        if result1:
                            k12GO_dict[x] = result1.groups()
                            #print(result1.groups())
                    else: pass
    else: print ("========I DID NOT FIND A TABLE FOR========== :: " + x)

========I DID NOT FIND A TABLE FOR========== :: s0001
========I DID NOT FIND A TABLE FOR========== :: b4104


In [30]:
k12splits = {}

for x, y in k12GO_dict.items():
    for m in y:
        result = re.split('\\xa0\\xa0', m)
        b = []        
        for go in result:
            result1 = re.split('eco', go)
            c = []
            for term in result1:
                result2 = re.split('\\n' , term)
                #print (result2 , " is to " , x)
                c.append(result2[:])
            b.append(c)
            #print ("The list b is :",  b)
            #print (b , x)
        k12splits[x] = b
    print ("I have added",x , "to the dictionary")

I have added b2551 to the dictionary
I have added b0870 to the dictionary
I have added b3368 to the dictionary
I have added b2436 to the dictionary
I have added b3500 to the dictionary
I have added b0945 to the dictionary
I have added b4467 to the dictionary
I have added b4468 to the dictionary
I have added b2979 to the dictionary
I have added b3916 to the dictionary
I have added b1260 to the dictionary
I have added b1261 to the dictionary
I have added b3426 to the dictionary
I have added b2242 to the dictionary
I have added b2243 to the dictionary
I have added b2241 to the dictionary
I have added b4054 to the dictionary
I have added b3770 to the dictionary
I have added b3281 to the dictionary
I have added b1692 to the dictionary
I have added b0243 to the dictionary
I have added b1613 to the dictionary
I have added b0414 to the dictionary
I have added b3041 to the dictionary
I have added b0025 to the dictionary
I have added b0844 to the dictionary
I have added b3812 to the dictionary
I

In [31]:
#first join all the lists together if there is many in the values of pa14splits
k12splits_joined = {}
for key, value in k12splits.items():
    value1 = [j for i in value for j in i]
    value2 = [m for n in value1 for m in n]
    #print (value2)
    k12splits_joined[key] = value2
#generate the numbers of genes that fall into the different ontologies
#now within the kegg ontologies for the pa14 model

#initiate a list of terms to be extracted from the dictionary, this will ensure that only terms present in the list will be used as categories for the genes to actually fall into
k12go_terms_kegg = []
for key,value in k12splits_joined.items():
    #print (value)
    for term in value:
        #print (term)
        if term in k12go_terms_kegg:
            pass
        else:
            k12go_terms_kegg.append(term)
            
#print (go_terms_kegg)

#now genreate the list of genes that correspond to each go term
#finally, count the number of members in each list, this is the number of genes that has a distinct ontology.
k12go_term_numbers_kegg= {}
for term in k12go_terms_kegg:
    #print (term , "=========================")
    k12go_list_each_ont = []
    for key, value in k12splits_joined.items():
        #print(value)

        if term in value:
            #print (term, key)
            k12go_list_each_ont.append(key)
                #print('yes daddy')
        else:
            pass
    #print (go_list_each_ont)
    k12go_term_numbers_kegg[term] = len(k12go_list_each_ont)
print (k12go_term_numbers_kegg)

{'': 1109, '00260': 36, 'Glycine, serine and threonine metabolism': 36, '00460': 7, 'Cyanoamino acid metabolism': 7, '00630': 40, 'Glyoxylate and dicarboxylate metabolism': 40, '00670': 13, 'One carbon pool by folate': 13, '00680': 30, 'Methane metabolism': 30, '01100': 806, 'Metabolic pathways': 806, '01110': 291, 'Biosynthesis of s': 291, 'ndary metabolites': 294, '01120': 241, 'Microbial metabolism in diverse environments': 241, '01130': 202, 'Biosynthesis of antibiotics': 202, '01200': 103, 'Carbon metabolism': 103, '01230': 114, 'Biosynthesis of amino acids': 114, '00860': 21, 'Porphyrin and chlorophyll metabolism': 21, 'Module': 409, '_M00846': 7, 'Siroheme biosynthesis, glutamate => siroheme': 7, '_M00121': 11, 'Heme biosynthesis, plants and bacteria, glutamate => heme': 11, '00480': 16, 'Glutathione metabolism': 16, '00240': 49, 'Pyrimidine metabolism': 49, '_M00051': 8, 'Uridine monophosphate biosynthesis, glutamine (+ PRPP) => UMP': 8, '00010': 40, 'Glycolysis / Gluconeogenes

In [34]:
#save all the organisms in data frame for csv

import pandas as pd
kegg_pa14_df = pd.DataFrame.from_dict(pa14go_term_numbers_kegg, orient = 'index')
kegg_pa14_df.to_csv("../data/pa14_gene_ontologies_figure_1.csv")

kegg_pao1_df = pd.DataFrame.from_dict(pao1go_term_numbers_kegg, orient = 'index')
kegg_pao1_df.to_csv("../data/pao1_gene_ontologies_figure_1.csv")

kegg_k12_df = pd.DataFrame.from_dict(k12go_term_numbers_kegg, orient = 'index')
kegg_k12_df.to_csv("../data/k12_gene_ontologies_figure_1.csv")

In [41]:
len(pa14.genes)

525

In [42]:
draft = cobra.io.read_sbml_model("../results/reconstructions/psy_draft.sbml")

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy,information is not parsed
SBML package 'render' not supported by cobrapy,information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00443_c0 "ABEE_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd02920_c0 "2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydropteridinediphosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00012_c0 "PPi_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00067_c0 "H_plus__c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00683_c0 "Dihydropteroate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00002_c0 "ATP_c0">
Use of the species charge attribute is discouraged, use fbc:charge i

In [47]:
draft.reactions.rxn01807_c0

Reaction identifier,rxn01807_c0
Name,ATP_pyridoxal_5_phosphotransferase_c0
Memory address,0x07f9b1cd1b3c8
Stoichiometry,cpd00002_c0 + cpd00419_c0 <=> cpd00008_c0 + cpd00067_c0 + cpd00493_c0 ATP_c0 + PM_c0 <=> ADP_c0 + H_plus__c0 + Pyridoxamine_phosphate_c0
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [5]:
import re
f = open('../data/modelseed_data/draft_gene_associations.txt', "r")
draft_gene_txt = f.read()
draft_genes = re.findall('PSPTO_'r'[0-9]+', draft_gene_txt)


In [9]:
#this is to gather information from the kegg website, using the number list generated in cell 4. 

GO_dict_draft = {}
url = "https://www.genome.jp/dbget-bin/www_bget?pst:"
for x in draft_genes:
    url_full = url + str(x)
    opened_url = urllib.request.urlopen(url_full)
    soup = bs4.BeautifulSoup(opened_url)
#print (soup)
    table = soup.find("table")
    if table != None:
        headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]
  # print (headings)
        for row in table.find_all("tr")[1:]:
            dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
   #print (dataset)
            if "Pathway" in dataset:
       #datasets.append(dataset)
                for key, value in dataset.items():
                    if "Pathway" in value:
           #print (value)
                        result = re.search('(?s)Pathway(.*)Module', value)
                        if result:
                            GO_dict_draft[x] = result.groups()
                           #print(result.groups())
                        result1 = re.search('(?s)Pathway(.*)Brite', value)
                        if result1:
                            GO_dict_draft[x] = result1.groups()
                            #print(result1.groups())
                    else: pass
    else: print ("========I DID NOT FIND A TABLE FOR========== :: " + x)

In [10]:
draft_ont = {}

for x, y in GO_dict_draft.items():
    for m in y:
        result = re.split('\\xa0\\xa0', m)
        kegg_b = []        
        for go in result:
            result1 = re.split('pst', go)
            kegg_c = []
            for term in result1:
                result2 = re.split('\\n' , term)
                #print (result2 , " is to " , x)
                kegg_c.append(result2[:])
            kegg_b.append(kegg_c)
            #print ("The list b is :",  b)
            #print (b , x)
        draft_ont[x] = kegg_b
    print ("I have added", x , "to the dictionary")
joined_draft_ont = {}
for key, value in draft_ont.items():
    value1 = [j for i in value for j in i]
    value2 = [m for n in value1 for m in n]
    #print (value2)
    joined_draft_ont["PSPTO_" + str(key)] = value2
print (joined_draft_ont)

#initiate a list of terms to be extracted from the dictionary, this will ensure that only terms present in the list will be used as categories for the genes to actually fall into
go_terms_draft = []
for key,value in joined_draft_ont.items():
    #print (value)
    for term in value:
        #print (term)
        if term in go_terms_draft:
            pass
        else:
            go_terms_draft.append(term)
            
#print (go_terms_kegg)

#now genreate the list of genes that correspond to each go term
#finally, count the number of members in each list, this is the number of genes that has a distinct ontology.
go_term_numbers_draft= {}
for term in go_terms_draft:
    #print (term , "=========================")
    go_list_each_ont = []
    for key, value in joined_draft_ont.items():
        #print(value)

        if term in value:
            #print (term, key)
            go_list_each_ont.append(key)
                #print('yes daddy')
        else:
            pass
    #print (go_list_each_ont)
    go_term_numbers_draft[term] = len(go_list_each_ont)
print (go_term_numbers_draft)

import pandas as pd
draft_pto_df = pd.DataFrame.from_dict(go_term_numbers_draft, orient = 'index')
print (draft_pto_df)
draft_pto_df.to_csv("../data/draft_pto_gene_ontologies_figure_1.csv")

I have added PSPTO_4496 to the dictionary
I have added PSPTO_1041 to the dictionary
I have added PSPTO_1748 to the dictionary
I have added PSPTO_5035 to the dictionary
I have added PSPTO_1131 to the dictionary
I have added PSPTO_3295 to the dictionary
I have added PSPTO_0494 to the dictionary
I have added PSPTO_5178 to the dictionary
I have added PSPTO_1421 to the dictionary
I have added PSPTO_1749 to the dictionary
I have added PSPTO_3668 to the dictionary
I have added PSPTO_4408 to the dictionary
I have added PSPTO_1356 to the dictionary
I have added PSPTO_4097 to the dictionary
I have added PSPTO_4098 to the dictionary
I have added PSPTO_0325 to the dictionary
I have added PSPTO_3721 to the dictionary
I have added PSPTO_1136 to the dictionary
I have added PSPTO_1430 to the dictionary
I have added PSPTO_0239 to the dictionary
I have added PSPTO_2304 to the dictionary
I have added PSPTO_5005 to the dictionary
I have added PSPTO_3860 to the dictionary
I have added PSPTO_3862 to the dic